# Introduction

## Explanation 1.1: Linear Programming

In the offline scenario, the total utility of the optimal task assignment, which is marked in red in TABLE 1, is 17.

|    | t1 | t2 | t3 | t4 | t5 | t6 |
|----|----|----|----|----|----|----|
| w1(1) |  7  |  1   |    |    |    |    |
| w2(3) |    |     |  1  |    |    |    |
| w3(2) |    |    |   9 |  1  |  1  |  1  |

In [15]:
from scipy.optimize import linprog
import numpy as np

c = -np.array([7, 1, 0, 0, 0, 0,
               0, 0, 1, 0, 0, 0,
               0, 0, 9, 1, 1, 1
               ])

A_ub = [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        ]

b_ub = [1, 3, 2, 1, 1, 1, 1, 1, 1]

bounds = [(0, 1) for _ in range(18)]

res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=1)

res.fun

-17.0

In [16]:
res.x

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  1.])

## Explanation 1.2

The GOMA problem, which generalizes the OMWBM problem, mainly differs from the OMWBM problem in that both the tasks and the workers are dynamic. 

| Timestamp | 0  | 1  | 2 | 7 | 8 | 9 | 9 | 15 | 18 |
|-----------|----|----|---|---|---|---|---|----|----|
| 1st order | t1 | t2 | w1  | w2  | t3  | t4  | w3  | t5   | t6   |
| 2nd order | w1 | t1   | t2  | t3  | w3  | t4  | w2  | t6   | t5   |

see Keynote.

# Problem Statement